In [29]:
### Install dependencies ###
!pip install keras numpy sklearn > /dev/null 2>&1

In [30]:
### Imports ###

# Keras
import keras.utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
# Processing
import numpy as numpy
# Used to encode labels like the monster classes to one-hot vectors
from sklearn.preprocessing import LabelEncoder
# Static typing ftw
from typing import List, cast

In [31]:
### Data processing ###

def labels_to_int(labels: List[str]) -> List[int]:
  """
  Transforms:
    ['Ghoul', 'Goblin', 'Ghoul', 'Ghoul', 'Ghost',
  to:
    [1, 2, 1, 1, 0,
  """
  return LabelEncoder().fit(labels).transform(labels)

def load_features(file_name: str) -> List:
  """
  Loads the features we are using for inferring the target.
  Those features are: bone_length  rotting_flesh  hair_length  has_soul
  TODO: Add 'color'
  
  The csv files look like this:
  
  id  bone_length  rotting_flesh  hair_length  has_soul  color    type
  0     0.354512       0.350839     0.465761  0.781142  clear   Ghoul
  1     0.575560       0.425868     0.531401  0.439899  green  Goblin
  2     0.467875       0.354330     0.811616  0.791225  black   Ghoul
  """
  return numpy.genfromtxt('data/' + file_name,
                          skip_header=1,
                          delimiter=",",
                          usecols=(1,2,3,4))

def load_targets(file_name: str) -> List:
  """
  Loads the labels we are using for inferring the target ('Ghost', 'Goblin', 'Ghoul')
  and transforms them into a one-hot vector.
  So basically this:
    ['Ghoul', 'Goblin', 'Ghoul', 'Ghoul', 'Ghost', .....
  becomes:
    [[ 0.,  1.,  0.],
     [ 0.,  0.,  1.],
     [ 0.,  1.,  0.],
  """
  labels = numpy.genfromtxt('data/' + file_name,
                            skip_header=1,
                            delimiter=",",
                            usecols=6,
                            dtype=str)
  return keras.utils.to_categorical(labels_to_int(labels))

In [32]:
### Load the data ###

x_train = load_features('train.csv')
y_train = load_targets('train.csv')
x_test = load_features('test.csv')
# y_test = load_targets('test.csv')

In [33]:
### Build the models ###

FEATURES = ['bone_length', 'rotting_flesh', 'hair_length', 'has_soul']
INPUT_DIM = len(FEATURES)
TARGET_VARIABLE = ['type']
VALID_TYPES = ['Ghost', 'Goblin', 'Ghoul']
OUTPUT_DIM = len(VALID_TYPES)

model = Sequential()

# Basic model inspired from:
# https://keras.io/getting-started/sequential-model-guide/
# section: "Multilayer Perceptron (MLP) for multi-class softmax classification"
#
# Play around with:
# * dropout
# * learning algorithm and its parameters
# * activation
# * learning rate
model.add(Dense(10, activation='relu', input_dim=INPUT_DIM))
model.add(Dense(OUTPUT_DIM, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

model.fit(x_train, y_train,
          epochs=20,
          batch_size=128)
#score = model.evaluate(x_test, y_test, batch_size=128)
#print "Score is " + score

Epoch 1/20
371/371 [==============================] - 0s - loss: 1.0918 - acc: 0.3477     
Epoch 2/20
371/371 [==============================] - 0s - loss: 1.0875 - acc: 0.3477     
Epoch 3/20
371/371 [==============================] - 0s - loss: 1.0825 - acc: 0.3477     
Epoch 4/20
371/371 [==============================] - 0s - loss: 1.0760 - acc: 0.3477     
Epoch 5/20
371/371 [==============================] - 0s - loss: 1.0694 - acc: 0.3531     
Epoch 6/20
371/371 [==============================] - 0s - loss: 1.0633 - acc: 0.3612     
Epoch 7/20
371/371 [==============================] - 0s - loss: 1.0566 - acc: 0.3693     
Epoch 8/20
371/371 [==============================] - 0s - loss: 1.0512 - acc: 0.4178     
Epoch 9/20
371/371 [==============================] - 0s - loss: 1.0461 - acc: 0.4636     
Epoch 10/20
371/371 [==============================] - 0s - loss: 1.0416 - acc: 0.4852     
Epoch 11/20
371/371 [==============================] - 0s - loss: 1.0379 - acc: 0.5067   